In [553]:
import re
from collections import Counter, defaultdict
from functools import cache
from itertools import permutations, product
from math import gcd, prod
from typing import List

import black
import jupyter_black
from parse import parse

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)


def ints(text: str) -> List[int]:
    return [int(x) for x in re.findall("-?\d+", text)]


def first(iterable):
    return next(iter(iterable))

In [554]:
# Day 1: Report Repair
numbers = [int(x) for x in open("2020/1.txt").read().splitlines()]
for a, b in permutations(numbers, 2):
    if a + b == 2020:
        break
print(f"Part 1: {a*b}")  # 471019

for a, b, c in permutations(numbers, 3):
    if a + b + c == 2020:
        break
print(f"Part 2: {a*b*c}")  # 103927824

Part 1: 471019
Part 2: 103927824


In [555]:
# Day 2: Password Philosophy
def valid(line):
    lower, upper, char, password = parse("{:d}-{:d} {}: {}", line)
    return lower <= password.count(char) <= upper


def valid2(line):
    first, second, char, password = parse("{:d}-{:d} {}: {}", line)
    return ((password[first - 1] == char) + (password[second - 1] == char)) == 1


lines = open("2020/2.txt").read().splitlines()
print(f"Part 1: {sum(valid(line) for line in lines)}")  # 398
print(f"Part 2: {sum(valid2(line) for line in lines)}")  # 562

Part 1: 398
Part 2: 562


In [556]:
# Day 3: Toboggan Trajectory
def downhill(lines, right=3, down=1, tree="#"):
    return sum(
        1
        for row, line in enumerate(lines[::down])
        if line[(right * row) % len(line)] == tree
    )


def run(lines, slopes):
    return prod(downhill(lines, *slope) for slope in slopes)


lines = open("2020/3.txt").read().splitlines()
slopes = ((1, 1), (3, 1), (5, 1), (7, 1), (1, 2))
print(f"Part 1: {downhill(lines)}")  # 189
print(f"Part 2: {run(lines, slopes)}")  # 1718180100

Part 1: 189
Part 2: 1718180100


In [557]:
# Day 4: Passport Processing
def all_fields_present(passport):
    FIELDS_NEEDED = {"byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid"}
    fields = passport.split(" ")
    fields = {field.split(":")[0]: field.split(":")[1] for field in fields}
    return all(field in fields.keys() for field in FIELDS_NEEDED)


def valid(passport):
    def check_height(height):
        unit = height[-2:]
        value = first(ints(height))
        if unit == "cm":
            return 150 <= value <= 193
        elif unit == "in":
            return 59 <= value <= 76
        else:
            return False

    EYE_COLORS = {"amb", "blu", "brn", "grn", "gry", "hzl", "oth"}
    rules = {
        "byr": lambda year: 1920 <= int(year) <= 2002,
        "iyr": lambda year: 2010 <= int(year) <= 2020,
        "eyr": lambda year: 2020 <= int(year) <= 2030,
        "hgt": check_height,
        "hcl": lambda color: re.search("#[0-9a-f]{6}", color) is not None,
        "ecl": lambda color: color in EYE_COLORS,
        "pid": lambda number: re.search("^[0-9]{9}$", number) is not None,
        "cid": lambda x: True,
    }
    fields = passport.split(" ")
    fields = {field.split(":")[0]: field.split(":")[1] for field in fields}
    if all(rules[key](value) for key, value in fields.items()):
        return True


passports = open("2020/4.txt").read().strip().split("\n\n")
passports = [p.replace("\n", " ") for p in passports]
passports = [p for p in passports if all_fields_present(p)]
print(f"Part 1: {len(passports)}")
passports = [p for p in passports if valid(p)]
print(f"Part 2: {len(passports)}")

Part 1: 170
Part 2: 103


In [558]:
# Day 5: Binary Boarding
def seat_id(boarding_pass):
    row = boarding_pass[:7].replace("F", "0").replace("B", "1")
    column = boarding_pass[7:].replace("L", "0").replace("R", "1")
    return int(row, 2) * 8 + int(column, 2)


boarding_passes = open("2020/5.txt").read().splitlines()
seat_ids = sorted(seat_id(boarding_pass) for boarding_pass in boarding_passes)
print(f"Part 1: {max(seat_ids)}")
my_seat_id = first(
    (higher - 1) for lower, higher in zip(seat_ids, seat_ids[1:]) if higher - lower == 2
)
print(f"Part 2: {my_seat_id}")

Part 1: 991
Part 2: 534


In [559]:
# Day 6: Custom Customs
def unique_answers(groups):
    total = 0
    for group in groups:
        answers = set()
        for person in group:
            for question in person:
                answers.add(question)
        total += len(answers)
    return total


def all_yes_answers(groups):
    total = 0
    for group in groups:
        group_answers = Counter()
        for person in group:
            for question in person:
                group_answers += Counter(question)
        for answer, count in group_answers.items():
            if count == len(group):
                total += 1
    return total


groups = open("2020/6.txt").read().strip().split("\n\n")
groups = [group.splitlines() for group in groups]

print(f"Part 1: {unique_answers(groups)}")
print(f"Part 2: {all_yes_answers(groups)}")

Part 1: 6683
Part 2: 3122


In [560]:
# Day 7: Handy Haversacks
def parse_lines(lines):
    bags = defaultdict(lambda: list())
    parents = defaultdict(lambda: set())
    for line in lines:
        source, destinations = line.split(" bags contain ")
        for destination in destinations.split(", "):
            if p := parse("{:d} {} ba{}", destination):
                amount, bag, _ = p
                bags[source] += [(amount, bag)]
                parents[bag] |= {source}
            else:
                bags[source] = []

    return bags, parents


def can_contain(bag_type="shiny gold"):
    result = parents[bag_type].copy()
    for parent in parents[bag_type]:
        result |= can_contain(parent)
    return result


def number_of_bags(bag_type="shiny gold"):
    "Return number of bags contained in this bag"
    return sum(
        num_child + num_child * number_of_bags(child)
        for num_child, child in bags[bag_type]
    )


lines = open("2020/7.txt").read().splitlines()
bags, parents = parse_lines(lines)
print(f"Part 1: {len(can_contain())}")  # 197
print(f"Part 2: {number_of_bags()}")  # 85324

Part 1: 197
Part 2: 85324


In [561]:
# Day 8: Handheld Halting
def run_program(lines):
    "Returns (True, result) if program runs to completion, (False, result) if it ends up in an infinite loop"
    pc = acc = 0
    pcs = set()
    for _ in range(1000):
        instruction, value = parse("{} {:d}", lines[pc])
        match instruction:
            case "acc":
                acc += value
            case "jmp":
                pc += value - 1
        pc += 1
        if pc in pcs or pc >= len(lines):
            return (pc >= len(lines), acc)
        pcs.add(pc)


def modified_programs(lines):
    modified_lines = lines.copy()
    for pc, line in enumerate(lines):
        instruction, value = parse("{} {}", lines[pc])
        if instruction == "jmp":
            modified_lines[pc] = "nop " + value
        elif instruction == "nop":
            modified_lines[pc] = "jmp " + value
        else:
            continue
        yield modified_lines.copy()
        modified_lines[pc] = lines[pc]


def part2(lines):
    for program in modified_programs(lines):
        (part2, result) = run_program(program)
        if part2:
            return result


lines = open("2020/8.txt").read().splitlines()
print(f"Part 1: {run_program(lines)[1]}")  # 1675
print(f"Part 2: {part2(lines)}")  # 1532

Part 1: 1675
Part 2: 1532


In [562]:
# Day 9: Encoding Error
def valid(position, numbers, length):
    for a, b in permutations(numbers[position - length : position], 2):
        if numbers[position] == a + b:
            return True
    return False


def first_invalid(numbers, length=25):
    return first(
        numbers[pos]
        for pos in range(length, len(numbers))
        if not valid(pos, numbers, length)
    )


def target_sequence(target, numbers):
    for start in range(len(numbers)):
        for end in range(start + 1, len(numbers)):
            if sum(numbers[start:end]) == target:
                return numbers[start:end]


lines = open("2020/9.txt").read().splitlines()
numbers = [int(x) for x in lines]
invalid = first_invalid(numbers)
sequence = target_sequence(invalid, numbers)
print(f"Part 1: {invalid}")
print(f"Part 2: {min(sequence) + max(sequence)}")

KeyboardInterrupt: 

In [ ]:
# Day 10: Adapter Array
def part1(lines):
    adapters = sorted(int(x) for x in lines)
    ones = threes = 1  # First and last jump
    for a, b in zip(adapters, adapters[1:]):
        if a + 1 == b:
            ones += 1
        else:
            threes += 1
    return ones * threes


def part2(lines):
    # Number of combinations are x1 * x2 * ... * xn where x1 is the number
    # of combinations for a sequence of 1-jumps.
    # Longest sequence of 1-jumps in input data is 5.
    numbers = sorted(int(x) for x in lines)
    combinations = {1: 1, 2: 1, 3: 2, 4: 4, 5: 7}  # Done by hand
    sequence_length = 2  # first sequence includes 0
    result = 1
    for a, b in zip(numbers, numbers[1:]):
        if a + 1 == b:
            sequence_length += 1
        else:
            result *= combinations[sequence_length]
            sequence_length = 1
    result *= combinations[sequence_length]
    return result


lines = open("2020/10.txt").read().splitlines()
print(f"Part 1: {part1(lines)}")  # 2277
print(f"Part 2: {part2(lines)}")  # 37024595836928

Part 1: 2277
Part 2: 37024595836928


In [ ]:
# Alternative implementations
@cache
def arrangements(jolts, prev) -> int:
    "The number of arrangements that go from prev to the end of `jolts`."
    first, rest = jolts[0], jolts[1:]
    if first - prev > 3:
        return 0
    elif not rest:
        return 1
    else:
        return arrangements(rest, first) + arrangements(rest, prev)


jolts = tuple(sorted(int(x) for x in lines))
# print(arrangements(jolts, 0))

# Depth first + memoization
def dfc(D, v, M={}):
    "Memoized depth first counter"
    if v in M:
        return M[v]
    elif D[v]:
        M[v] = sum(dfc(D, x, M) for x in D[v])
        return M[v]
    else:
        return 1


jolts = (0,) + tuple(sorted(int(x) for x in lines))
dag = {x: {y for y in range(x + 1, x + 4) if y in jolts} for x in jolts}
# print(dfc(dag, 0))

# Fibonacci-like dynamic programming
adapters = tuple(sorted(int(x) for x in lines))
answer = defaultdict(int)
answer[0] = 1
for a in adapters:
    answer[a] = answer[a - 1] + answer[a - 2] + answer[a - 3]
# print(answer[max(adapters)])

In [ ]:
# Day 11: Seating Systems
def close_neighbors(grid, row, col):
    for (dr, dc) in DIRECTIONS:
        if (row + dr, col + dc) in grid.keys():
            yield (row + dr, col + dc)


def diagonal_neighbors(grid, start_row, start_col, floor="."):
    for (dr, dc) in DIRECTIONS:
        row = start_row + dr
        col = start_col + dc
        while (row, col) in grid.keys() and grid[(row, col)] == floor:
            row += dr
            col += dc
        if (row, col) in grid.keys():
            yield (row, col)


def occupied_neighbors(grid, neighbors, row, col, occupied="#"):
    return sum(grid[point] == occupied for point in neighbors(grid, row, col))


def occupied_seats(grid, occupied="#"):
    return sum(seat == occupied for seat in grid.values())


def seating_round(grid, neighbors, max_occupants, empty="L", occupied="#"):
    new_grid = {}
    for (row, col), seat in grid.items():
        if seat == empty and occupied_neighbors(grid, neighbors, row, col) == 0:
            new_grid[(row, col)] = occupied
        elif (
            seat == occupied
            and occupied_neighbors(grid, neighbors, row, col) >= max_occupants
        ):
            new_grid[(row, col)] = empty
        else:
            new_grid[(row, col)] = grid[(row, col)]
    return new_grid


def occupied_seats_when_stable(grid, neighbors, max_occupants=4):
    for _ in range(1000):
        prev_grid = seating_round(grid, neighbors, max_occupants)
        if prev_grid == grid:
            break
        grid = prev_grid
    return occupied_seats(grid)


# fmt:off
DIRECTIONS = ((-1, -1), (-1, 0), (-1, 1),
              ( 0, -1),          ( 0, 1),
              ( 1, -1), ( 1, 0), ( 1, 1))
# fmt:on

lines = open("2020/11.txt").read().splitlines()
grid = {
    (row, col): lines[row][col]
    for row, col in product(range(len(lines)), range(len(lines[0])))
}

print(f"Part 1: {occupied_seats_when_stable(grid, close_neighbors)}")  # 2316, 4s
print(f"Part 2: {occupied_seats_when_stable(grid, diagonal_neighbors, 5)}")  # 2128, 4s

Part 1: 2316
Part 2: 2128


In [ ]:
# Day 12: Rain Risk
# dx, dy for all directions. Turning right is equivalent to moving to next position in
# DIRECTIONS and turning left is previous position
EAST, SOUTH, WEST, NORTH = 0, 1, 2, 3
RIGHT, LEFT = +1, -1
DIRECTIONS = [(1, 0), (0, -1), (-1, 0), (0, 1)]  # E, S, W, N


class Ship:
    direction = EAST
    x = 0
    y = 0

    def forward(self, steps):
        dx, dy = DIRECTIONS[self.direction]
        for _ in range(steps):
            self.x += dx
            self.y += dy

    def turn(self, dir, degrees):
        assert degrees in (0, 90, 180, 270)
        turns = degrees // 90
        for _ in range(turns):
            self.direction += dir
            self.direction %= 4

    def move(self, dir, steps):
        dx, dy = DIRECTIONS[dir]
        for _ in range(steps):
            self.x += dx
            self.y += dy


class WaypointShip(Ship):
    waypoint_x = 10
    waypoint_y = 1

    def forward(self, steps):
        self.x += self.waypoint_x * steps
        self.y += self.waypoint_y * steps

    def turn(self, dir, degrees):
        assert degrees in (0, 90, 180, 270)
        turns = degrees // 90
        if dir == LEFT:
            turns = 4 - turns
        for _ in range(turns):
            # One right turn
            self.waypoint_x, self.waypoint_y = self.waypoint_y, -self.waypoint_x

    def move(self, dir, steps):
        dx, dy = DIRECTIONS[dir]
        for _ in range(steps):
            self.waypoint_x += dx
            self.waypoint_y += dy


def sail(lines):
    ship = Ship()
    waypoint_ship = WaypointShip()
    for line in lines:
        action, number = line[0], int(line[1:])
        match action:
            case "N":
                ship.move(NORTH, number)
                waypoint_ship.move(NORTH, number)
            case "S":
                ship.move(SOUTH, number)
                waypoint_ship.move(SOUTH, number)
            case "E":
                ship.move(EAST, number)
                waypoint_ship.move(EAST, number)
            case "W":
                ship.move(WEST, number)
                waypoint_ship.move(WEST, number)
            case "L":
                ship.turn(LEFT, number)
                waypoint_ship.turn(LEFT, number)
            case "R":
                ship.turn(RIGHT, number)
                waypoint_ship.turn(RIGHT, number)
            case "F":
                ship.forward(number)
                waypoint_ship.forward(number)
    part1 = abs(ship.x) + abs(ship.y)
    part2 = abs(waypoint_ship.x) + abs(waypoint_ship.y)
    return part1, part2


lines = open("2020/12.txt").read().splitlines()
part1, part2 = sail(lines)
print("Part 1:", part1) # 858
print("Part 2:", part2) # 39140

Part 1: 858
Part 2: 39140


In [667]:
# Day 13: Shuttle Search
def earliest_bus(earliest_time, buses):
    for departure_time in range(earliest_time, earliest_time + max(buses)):
        for bus in buses:
            if gcd(departure_time, bus) == bus:
                return (departure_time - earliest_time) * bus


# From https://math.stackexchange.com/questions/2218763/how-to-find-lcm-of-two-numbers-when-one-starts-with-an-offset
def combine_phased_rotations(a_period, a_phase, b_period, b_phase):
    """Combine two phased rotations into a single phased rotation

    Returns: combined_period, combined_phase

    The combined rotation is at its reference point if and only if both a and b
    are at their reference points.
    """
    gcd, s, t = extended_gcd(a_period, b_period)
    phase_difference = a_phase - b_phase
    pd_mult, pd_remainder = divmod(phase_difference, gcd)
    if pd_remainder:
        raise ValueError("Rotation reference points never synchronize.")

    combined_period = a_period // gcd * b_period
    combined_phase = (a_phase - s * pd_mult * a_period) % combined_period
    return combined_period, combined_phase


def extended_gcd(a, b):
    """Extended Greatest Common Divisor Algorithm

    Returns:
        gcd: The greatest common divisor of a and b.
        s, t: Coefficients such that s*a + t*b = gcd

    Reference:
        https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm#Pseudocode
    """
    old_r, r = a, b
    old_s, s = 1, 0
    old_t, t = 0, 1
    while r:
        quotient, remainder = divmod(old_r, r)
        old_r, r = r, remainder
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t

    return old_r, old_s, old_t


def earliest_timestamp(buses):
    # Combine one pair of period phase until only one remains
    period, phase = buses[0], 0
    for offset, number in buses.items():
        period, phase = combine_phased_rotations(period, phase, number, offset)
    return period - phase


lines = open("2020/13.txt").read().splitlines()
earliest_time = int(lines[0])
buses = {pos: int(x) for pos, x in enumerate(lines[1].split(",")) if x != "x"}
print(f"Part 1: {earliest_bus(earliest_time, buses.values())}")  # 161
print(f"Part 2: {earliest_timestamp(buses)}")  # 213890632230818

Part 1: 161
Part 2: 213890632230818


In [679]:
# Much simpler solution from Peter Norvig for day 13, part 2
def part2(buses):
    """Find the time where all the buses arrive at the right offsets.

    For each bus id, we want to find a time where we get that id right, then step the time by a multiple of all the ids encountered so far:
    """

    def wait(id, t):
        "How long you have to wait from t for bus id."
        return 0 if t % id == 0 else id - t % id

    time = 0
    step = 1
    for offset, bus in buses.items():
        while wait(bus, time + offset):
            time += step
        step *= bus
    return time

In [743]:
# Day 14: Docking Data
def run_program(lines):
    memory = {}
    for line in lines:
        if p := parse("mask = {}", line):
            zeroes = ""  # value = value and zeroes
            ones = ""  # value = value or ones
            for bit in p[0]:
                if bit == "X":
                    ones += "0"
                    zeroes += "1"
                else:
                    ones += bit
                    zeroes += bit
            ones = int(ones, 2)
            zeroes = int(zeroes, 2)
        elif p := parse("mem[{:d}] = {:d}", line):
            address, value = p
            value = value & zeroes | ones
            memory[address] = value
    return sum(memory.values())


def addresses(address, mask):
    def pattern(address, mask):
        result = ""
        address = format(address, "036b")  # zero padded 36-bit binary
        for m, a in zip(mask, address):
            if m == "0":
                result += a
            else:
                result += m
        return result

    mask = pattern(address, mask)
    repeat = mask.count("X")
    for bit_pattern in product("01", repeat=repeat):
        result = mask
        for bit in bit_pattern:
            result = result.replace("X", bit, 1)
        yield int(result, 2)


def run_program2(lines):
    memory = {}
    for line in lines:
        if p := parse("mask = {}", line):
            mask = p[0]
        elif p := parse("mem[{:d}] = {:d}", line):
            address, value = p
            for target in addresses(address, mask):
                memory[target] = value
    return sum(memory.values())


lines = open("2020/14.txt").read().splitlines()
print(f"Part 1: {run_program(lines)}")  # 8332632930672
print(f"Part 2: {run_program2(lines)}")  # 4753238784664

Part 1: 8332632930672


UnboundLocalError: local variable 'result' referenced before assignment